<img src="./images/logo.png" alt="utech academy">

<div class="alert">
<div style="direction:ltr;text-align:left;font-family:Tahoma; font-size:16pt">Performance Evaluation with Resampling
</div>
</div>

In [43]:
import cv2
import numpy as np
from scipy import io


def load_hoda(dataset_dir, training_sample_size=1000, test_sample_size=200, size=5):
    # Load the dataset
    trs = training_sample_size
    tes = test_sample_size
    dataset = io.loadmat(dataset_dir)

    # Split training and test set
    X_train_orginal = np.squeeze(dataset['Data'][:trs])
    y_train = np.squeeze(dataset['labels'][:trs])
    X_test_original = np.squeeze(dataset['Data'][trs:trs+tes])
    y_test = np.squeeze(dataset['labels'][trs:trs+tes])

    # Resize the dataset
    X_train_5by5 = [cv2.resize(img, dsize=(size, size)) for img in X_train_orginal]
    X_test_5by_5 = [cv2.resize(img, dsize=(size, size)) for img in X_test_original]
    
    # Reshape the dataset
    X_train = [x.reshape(size*size) for x in X_train_5by5]
    X_test = [x.reshape(size*size) for x in X_test_5by_5]
    
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

In [44]:
# Load the dataset
Data, labels, _, _ = load_hoda(dataset_dir='./datasets/hoda.mat')

In [45]:
from sklearn.model_selection import train_test_split

X, X_test, Y, y_test = train_test_split(Data, labels, test_size=0.1, random_state=42)

In [46]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

In [47]:
X_train.shape

(810, 25)

In [48]:
X_val.shape

(90, 25)

In [49]:
X_test.shape

(100, 25)

In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
for k in range(1, 21, 2):
    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(X_train, y_train)
    print(k, ':', neigh.score(X_val, y_val))

1 : 0.8666666666666667
3 : 0.8777777777777778
5 : 0.8888888888888888
7 : 0.8888888888888888
9 : 0.8777777777777778
11 : 0.8555555555555555
13 : 0.8666666666666667
15 : 0.8555555555555555
17 : 0.8555555555555555
19 : 0.8777777777777778


In [10]:
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
print(neigh.score(X_test, y_test))

0.91


In [17]:
from sklearn.model_selection import KFold

n_split = 5
cv = KFold(n_splits=n_split)
fold_count = 0
acc = []

for train_idx, test_idx in cv.split(Data, labels): 
    fold_count += 1

    x_train, x_test = Data[train_idx], Data[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(x_train, y_train)

    score = knn.score(x_test, y_test)
    print('Test accuracy on fold ' + str(fold_count) + ' :', score)
    acc.append(score)

Test accuracy on fold 1 : 0.905
Test accuracy on fold 2 : 0.92
Test accuracy on fold 3 : 0.9
Test accuracy on fold 4 : 0.855
Test accuracy on fold 5 : 0.86


In [18]:
acc = np.array(acc)
print("Accuracy: %0.5f (+/- %0.5f)" % (acc.mean(), acc.std()))

Accuracy: 0.88800 (+/- 0.02581)


In [32]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
acc = []
n = 100

for train_index, test_index in loo.split(Data[:n]):
    x_train, x_test = Data[:n][train_index], Data[:n][test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(x_train, y_train)

    score = knn.score(x_test, y_test)
    print('Test accuracy on sample ' + str(test_index) + ' :', score)
    acc.append(score)

Test accuracy on sample [0] : 1.0
Test accuracy on sample [1] : 1.0
Test accuracy on sample [2] : 1.0
Test accuracy on sample [3] : 0.0
Test accuracy on sample [4] : 0.0
Test accuracy on sample [5] : 1.0
Test accuracy on sample [6] : 1.0
Test accuracy on sample [7] : 1.0
Test accuracy on sample [8] : 1.0
Test accuracy on sample [9] : 1.0
Test accuracy on sample [10] : 1.0
Test accuracy on sample [11] : 0.0
Test accuracy on sample [12] : 1.0
Test accuracy on sample [13] : 0.0
Test accuracy on sample [14] : 1.0
Test accuracy on sample [15] : 1.0
Test accuracy on sample [16] : 1.0
Test accuracy on sample [17] : 1.0
Test accuracy on sample [18] : 1.0
Test accuracy on sample [19] : 1.0
Test accuracy on sample [20] : 0.0
Test accuracy on sample [21] : 1.0
Test accuracy on sample [22] : 0.0
Test accuracy on sample [23] : 0.0
Test accuracy on sample [24] : 1.0
Test accuracy on sample [25] : 1.0
Test accuracy on sample [26] : 0.0
Test accuracy on sample [27] : 1.0
Test accuracy on sample [28] :

In [33]:
acc = np.array(acc)
print("Accuracy: %0.5f (+/- %0.5f)" % (acc.mean(), acc.std()))

Accuracy: 0.70000 (+/- 0.45826)


In [41]:
acc = []

for i in range(100):
    X, X_test, Y, y_test = train_test_split(Data, labels, test_size=0.1)
    
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X, Y)

    score = knn.score(X_test, y_test)
    print('Test accuracy in turn ' + str(i) + ' :', score)
    acc.append(score)

Test accuracy in turn 0 : 0.85
Test accuracy in turn 1 : 0.9
Test accuracy in turn 2 : 0.88
Test accuracy in turn 3 : 0.94
Test accuracy in turn 4 : 0.92
Test accuracy in turn 5 : 0.93
Test accuracy in turn 6 : 0.91
Test accuracy in turn 7 : 0.9
Test accuracy in turn 8 : 0.94
Test accuracy in turn 9 : 0.93
Test accuracy in turn 10 : 0.9
Test accuracy in turn 11 : 0.87
Test accuracy in turn 12 : 0.91
Test accuracy in turn 13 : 0.89
Test accuracy in turn 14 : 0.87
Test accuracy in turn 15 : 0.87
Test accuracy in turn 16 : 0.86
Test accuracy in turn 17 : 0.93
Test accuracy in turn 18 : 0.9
Test accuracy in turn 19 : 0.89
Test accuracy in turn 20 : 0.83
Test accuracy in turn 21 : 0.91
Test accuracy in turn 22 : 0.94
Test accuracy in turn 23 : 0.91
Test accuracy in turn 24 : 0.88
Test accuracy in turn 25 : 0.83
Test accuracy in turn 26 : 0.9
Test accuracy in turn 27 : 0.91
Test accuracy in turn 28 : 0.85
Test accuracy in turn 29 : 0.92
Test accuracy in turn 30 : 0.86
Test accuracy in turn 3

In [42]:
acc = np.array(acc)
print("Accuracy: %0.5f (+/- %0.5f)" % (acc.mean(), acc.std()))

Accuracy: 0.89380 (+/- 0.03212)


<div class="alert">
<div style="direction:ltr;text-align:left;font-family:B Tahoma"> Machine Learning Course
<br>Vahid Reza Khazaie<br>
</div>
<a href="https://www.linkedin.com/in/vahidrezakhazaie/">LinkedIn</a> - <a href="https://github.com/vrkh1996">GitHub</a>

</div>